In [1]:
import pandas as pd

In [2]:
df_words=pd.read_csv('./data/p1_weight_sorted.csv')

In [3]:
df_words

,words,weightage,a,b,c,d,e,f,g,h,...,q,r,s,t,u,v,w,x,y,z
0,esses,0.513706,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,sessa,0.503346,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,sasse,0.503346,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,asses,0.503346,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,sease,0.503299,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12967,muzzy,0.114539,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,1
12968,whizz,0.114493,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,1
12969,huzzy,0.111209,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,1,1
12970,buzzy,0.109158,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,1


# Writing filtering functions

In [4]:
# if word =='donor':
#     print(f'Dropping {word}')
# df.drop(df[df[letter]==1].index, inplace = True)
# df.drop(df[df[letter]==0].index, inplace = True)

In [5]:
inword=''

In [6]:
def not_in_word(df, letter,at):
    global inword
    print(f"Letter {letter} not in word")
    if letter in inword:
        print(f"Letter {letter} is in the word so not deleting")
        not_at(df,letter,at)
        return
    df.drop(df[df[letter]==1].index, inplace = True)
#     for word in df['words']:
#         if letter in word:
#             if word =='sewer':
#                 print(f'Dropping {word}')
#             i=(df[df.words==word].index)
#             df.drop(i,inplace=True)

In [7]:
def not_at(df, letter,at):
    global inword
    inword+=letter
    at=at-1
    print(f"Letter {letter} not at place {at}")
    df.drop(df[df[letter]==0].index, inplace = True)
    print(f"IN Shape: {df.shape}")
    for word in df['words']:
        if letter == word[at]:
            if word =='sewer':
                print(f'Dropping {word}')
            i=(df[df.words==word].index)
            df.drop(i,inplace=True)

In [8]:
def at(df, letter,at):
    global inword
    inword+=letter
    print(f"Letter {letter} is at place {at}")
    df.drop(df[df[letter]==0].index, inplace = True)
    print(f"IN Shape: {df.shape}")
    at=at-1
    for word in df['words']:
        if letter != word[at]:
            if word =='sewer':
                print(f'Dropping {word}')
            i=(df[df.words==word].index)
            df.drop(i,inplace=True)

# Writing Guess function

In [9]:
def guess(df,word,color):
    print(f"recived {word}")
    for i,letter in enumerate(word):
        if color[i]=='r':
            not_in_word(df,letter,i+1)
            print(f"Shape: {df.shape}")
        elif color[i]=='g':
            at(df,letter,i+1)
            print(f"Shape: {df.shape}")
        elif color[i]=='y':
            not_at(df,letter,i+1)
            print(f"Shape: {df.shape}")

In [10]:
df=None
def play():
    global df
    df=df_words.copy()
    for i in range(5):
        word=input("Enter Word: ")
        if word=='111':
            print("Congratulations...")
            break
            return
        color=input("Enter colors: ")
        guess(df,word,color)
        print(df['words'].head())

In [11]:
play()

Enter Word: 111
Congratulations...


In [12]:
from selenium import webdriver

In [13]:
import time

In [14]:
import keyboard

In [15]:
path="./chromedriver.exe"

In [37]:
# Global variables
inword=''
invalid_case=0
# i=0
df=df_words.copy()
valid=True
row_cnt=2

print(f"Shape: {df.shape}")

driver=webdriver.Chrome(path)
driver.get("https://www.wordleunlimited.com/")

while True:

    # Selecting the guess word
    if row_cnt==2:
        word='audio'
    else:
        if df.shape[0]==0:
            print("No words in Dataset")
            break
        print(df['words'].head())
        word=list(df['words'])[0]
        valid=True

    # Typing the guess word
    row=driver.find_element_by_xpath(f'//*[@id="root"]/div/div[1]/div[{row_cnt}]')
    keyboard.write(word, delay=0.05)
    keyboard.press_and_release('enter')
    
    # Detecting the response
    color=''
    for j in range(5):
        letter=driver.find_element_by_xpath(f'//*[@id="root"]/div/div[1]/div[{row_cnt}]/div[{j+1}]')
        if 'letter-absent' in letter.get_attribute("class"):
            color+='r'
        elif 'letter-elsewhere' in letter.get_attribute("class"):
            color+='y'
        elif 'letter-correct' in letter.get_attribute("class"):
            color+='g'
        else:
            # Chaecking if guess word is not a valid word
            valid=False
            invalid_case+=1
            print("Invalid word")
            for k in range(5):
                keyboard.press_and_release('backspace')
            i=(df[df.words==word].index)
            df.drop(i,inplace=True)
    
    # evaluating the word and guessing the next word
    if valid:
        row_cnt+=1
        print(f"Colors: {color}")
        if color=='ggggg':
            print("Congretulation...")
            break
        else:
            print(f'Guess {row_cnt-2}: {word}')
            guess(df,word,color)
#     i+=1
    if row_cnt-2>6:
        break

# driver.quit() # for closing browser after run

Shape: (12972, 28)
Colors: yryrr
Guess 1: audio
recived audio
Letter a not at place 0
IN Shape: (5330, 28)
Shape: (4593, 28)
Letter u not in word
Shape: (4045, 28)
Letter d not at place 2
IN Shape: (649, 28)
Shape: (538, 28)
Letter i not in word
Shape: (445, 28)
Letter o not in word
Shape: (385, 28)
79     eased
412    sards
413    eards
414    dares
415    dears
Name: words, dtype: object
Colors: ryrrg
Guess 2: eased
recived eased
Letter e not in word
Shape: (154, 28)
Letter a not at place 1
IN Shape: (154, 28)
Shape: (63, 28)
Letter s not in word
Shape: (26, 28)
Letter e not in word
Shape: (26, 28)
Letter d is at place 5
IN Shape: (26, 28)
Shape: (10, 28)
8198    lyard
9221    grand
9245    brand
9315    ydrad
9316    dryad
Name: words, dtype: object
Colors: rrgyg
Guess 3: lyard
recived lyard
Letter l not in word
Shape: (6, 28)
Letter y not in word
Shape: (3, 28)
Letter a is at place 3
IN Shape: (3, 28)
Shape: (3, 28)
Letter r not at place 3
IN Shape: (3, 28)
Shape: (2, 28)
Letter d 